# Empenhos de órgãos municipais do litoral leste do Ceará ligados à Cultura
Trabalho de Extração de dados

Renato Freitas

## Gastos Públicos no Setor Cultural


Onde | R\$ | 2011 | R\$ | 2018
---|--|---------|--|----------
Gastos nas três esferas | R\$ 7.1 bilhões || R\$ 9.1 bilhões|
Federal | R\$ 1.360.528 | 19.2\% | R\$ 1.925.133 | 21.1\%
Estadual | R\$ 2.270.552 | 32.0\% | R\$ 2.504.970 | 27.5\%
Municipal | R\$ 3.462.388 | 48.8\% | R\$ 4.690.086 | 51.4\%
Despesas dos municípios cearenses | - |3.8\%| - |4.3\%


Fonte: IBGE, Sistema de Informações e Indicadores Culturais. https://biblioteca.ibge.gov.br/visualizacao/livros/liv101687.pdf

## Objetivo
Gerar um dataset de empenhos de gastos com cultura dos 5 anos anteriores dos municípios do litoral leste do Ceará. 

## Fonte de Dados
Portal de Sitemas e Consultas Municípios - TCE. https://municipios.tce.ce.gov.br/

<img src="tcm.png" />

##  Como obter os dados.
Módulo Dados Abertos - via API.

Para obter os empenhos é preciso conhecer o código do município, do órgão e informar ano pretendido.
 - PASSO 1. OBTER OS CÓDIGOS DOS MUNICÍPIOS
 - PASSO 2. OBTER OS CÓDIGOS DOS ORGÃO (SECRETARIA DE CULTURA, POR EXEMPLO)
 - PASSO 3. OBTER OS EMPENHOS

### PASSO 1. OBTER OS CÓDIGOS DOS MUNICÍPIOS

In [1]:
# IMPORTANDO BIBLIOTECAS
import requests
from bs4 import BeautifulSoup
from datetime import date
import csv
import pandas as pd
import json
import logging
import time

In [2]:
# Cidades do Litoral Leste do Ceará, segundo: 
cidades_litoral_leste = [
    {"nome": "Aracati"},
    {"nome": "Fortim"},
    {"nome": "Beberibe"},
    {"nome": "Cascavel"},
    {"nome": "Icapui"},
    {"nome": "Itaicaba"},
    {"nome": "Jaguaruana"},
    {"nome": "Pindoretama"}  
]

In [3]:
# API do Tribunal de Contas do Estado do Ceará
api_tce = "https://api.tce.ce.gov.br/index.php/sim/1_0/"

In [4]:
# Municipios do TCE
url_tce_municipios = lambda nome: "{}municipios.json?nome_municipio={}".format(api_tce, nome)

for cidade in cidades_litoral_leste:
    response = requests.get(url_tce_municipios(cidade["nome"]))
    municipio = response.json()["rsp"]["_content"][0]
    if municipio["nome_municipio"] == cidade["nome"].upper():
        cidade["codigo_tce"] = municipio["codigo_municipio"]
    
cidades_litoral_leste

[{'nome': 'Aracati', 'codigo_tce': '014'},
 {'nome': 'Fortim', 'codigo_tce': '183'},
 {'nome': 'Beberibe', 'codigo_tce': '028'},
 {'nome': 'Cascavel', 'codigo_tce': '041'},
 {'nome': 'Icapui', 'codigo_tce': '072'},
 {'nome': 'Itaicaba', 'codigo_tce': '082'},
 {'nome': 'Jaguaruana', 'codigo_tce': '091'},
 {'nome': 'Pindoretama', 'codigo_tce': '136'}]

### PASSO 2. OBTER OS CÓDIGOS DOS ORGÃOS (SECRETARIAS DE CULTURA, POR EXEMPLO)

In [5]:
#https://api.tce.ce.gov.br/index.php/sim/1_0/unidades_orcamentarias?codigo_municipio=002&exercicio_orcamento=201800

ano_atual = date.today().year

# Obter todos os orgão nos últimos 5 anos
for cidade in cidades_litoral_leste:
    orgao = {}
    orgaos_cultura = []
    for ano in range(ano_atual - 5, ano_atual):
        
        cod_mun_tce = cidade["codigo_tce"]
        api_tce_orgaos = "{}unidades_orcamentarias.json?codigo_municipio={}&exercicio_orcamento={}00".format(api_tce, cod_mun_tce, ano)
    
        response = requests.get(api_tce_orgaos)
        orgaos = response.json()["rsp"]["_content"]
        
        for key in orgaos:
            NOME_ORG = key["nome_unidade"].upper()
            if NOME_ORG.find("CULTURA") != -1 and NOME_ORG.find("AGRICULTURA") == -1 and NOME_ORG.find("AQUICULTURA") == -1:
                orgaos_cultura.append([key["codigo_orgao"], key["nome_unidade"], str(ano)])
                
        cidade["orgaos"] = orgaos_cultura
        
cidades_litoral_leste

[{'nome': 'Aracati',
  'codigo_tce': '014',
  'orgaos': [['06', 'Fundo Municipal de Cultura - FMC', '2016'],
   ['06', 'Fundo Municipal de Cultura - FMC', '2017'],
   ['12', 'Sec. de Turismo e Cultura', '2018'],
   ['12', 'Fundo Mun. de Cultura', '2018'],
   ['12', 'Sec. de Turismo e Cultura', '2019'],
   ['12', 'Fundo Mun. de Cultura', '2019'],
   ['12', 'Sec. de Turismo e Cultura', '2020'],
   ['12', 'Fundo Mun. de Cultura', '2020']]},
 {'nome': 'Fortim',
  'codigo_tce': '183',
  'orgaos': [['20', 'Secretaria Munic. de Turismo e Cultura', '2016'],
   ['20', 'Secretaria Munic. de Turismo e Cultura', '2017'],
   ['20', 'Secretaria Munic. de Turismo e Cultura', '2018'],
   ['20', 'Secretaria Munic. de Turismo e Cultura', '2019'],
   ['20', 'Secretaria Munic. de Turismo e Cultura', '2020']]},
 {'nome': 'Beberibe',
  'codigo_tce': '028',
  'orgaos': [['14', 'Secretaria de Turismo e Cultura', '2016'],
   ['24', 'Fundo Municipal de Cultura', '2016'],
   ['14', 'Secretaria de Turismo e Cultu

### PASSO 3. OBTER OS EMPENHOS

In [6]:
#https://api.tce.ce.gov.br/index.php/sim/1_0/notas_empenhos?codigo_municipio=014&codigo_orgao=06&data_referencia_empenho=201101
def getUrlTceEmpenhos(cod_mun_ce, cod_org_tce, ano): 
    """Recebe o código do municipio, o código do orgão e o ano"""
    
    return """{}notas_empenhos.json?codigo_municipio={}&codigo_orgao={}&data_referencia_empenho={}01""".format(api_tce, cod_mun_ce, cod_org_tce, ano)

In [7]:
empenhos = []
for cidade in cidades_litoral_leste:
    print(cidade["nome"])
    
    print(len(cidade["orgaos"]), end="\n")
        
    for orgao in cidade["orgaos"]:
        url = getUrlTceEmpenhos(cidade["codigo_tce"], orgao[0], orgao[2])
        
        resposta = requests.get(url)
        empenhos.extend(resposta.json()["rsp"]["_content"])
        
len(empenhos)

Aracati
8
Fortim
5
Beberibe
10
Cascavel
0
Icapui
10
Itaicaba
0
Jaguaruana
5
Pindoretama
1


889

In [9]:
empenhos

[{'codigo_municipio': '014',
  'exercicio_orcamento': '201600',
  'codigo_orgao': '06',
  'codigo_unidade': '02  ',
  'data_emissao_empenho': '2016-01-04 00:00:00',
  'numero_empenho': '04010307',
  'data_referencia_empenho': '201601',
  'codigo_funcao': '13',
  'codigo_subfuncao': '392',
  'codigo_programa': '0064',
  'codigo_projeto_atividade': '2',
  'numero_projeto_atividade': '043',
  'numero_subprojeto_atividade': '0000',
  'codigo_elemento_despesa': '33903000',
  'modalidade_empenho': 'O',
  'descricao_empenho': 'Valor que se empenha p/ fazer face as despesas com AQUISICAO DE COLCHOES DESTINADOS A SER UTILIZADO PELOS POLICIAIS QUE VAO FAZER A SEGURANCA DO CARNAVAL CULTURAL 2016 NO MUNICIPIO DE ARACATI.',
  'valor_anterior_saldo_dotacao': '8500.0000',
  'valor_empenhado': '5980.0000',
  'valor_atual_saldo_dotacao': '2520.0000',
  'tipo_processo_licitatorio': 'F',
  'numero_documento_negociante': '41426966005050',
  'estado_empenho': 'OR',
  'numero_nota_anulacao': ' ',
  'data_em

## Agregar valor aos empenhos 
Obter situação da empresa

Site: https://serenata.ai/

Site|Github
:--:|:--:
<img src="serenata.png" width="600"/>|<img src="git.png" />

Site: https://receitaws.com.br/

ReceitaWS|Planos
:--:|:--:
<img src="receita.png" width=600>|<img src="receita-planos.png">

In [14]:
# Obter situacao de fornecedors
# Fonte: https://github.com/okfn-brasil/serenata-de-amor/blob/main/research/src/fetch_cnpj_info.py

LOGGER_NAME = 'fetch_cnpj'
logger = logging.getLogger(LOGGER_NAME)
def fetch_cnpj_info(cnpj, timeout=60):
    url = 'http://receitaws.com.br/v1/cnpj/%s' % cnpj
    try:
        result = requests.get(url,timeout=timeout)
#         print(result.status_code)
        if result.status_code == 200 and result.json()["status"] != 'ERROR':
            return result.json()
        elif result.status_code == 429:
            logger.debug('Sleeping 60 seconds to try again.')
            time.sleep(60)
            result = requests.get(url,timeout=timeout)
#             print("sleep 60 secs: ",result.status_code)
            return result.json()
        else:
            logger.debug(result.json()['message'])
            return result.json()['message']
    except requests.exceptions.Timeout as e:
        logger.debug(e)
    except requests.exceptions.ConnectionError as e:
        logger.debug(e)

In [15]:
ex_situacao = fetch_cnpj_info("29979036004219")
ex_situacao

{'atividade_principal': [{'text': 'Seguridade social obrigatória',
   'code': '84.30-2-00'}],
 'data_situacao': '03/11/2005',
 'tipo': 'FILIAL',
 'nome': 'INSTITUTO NACIONAL DO SEGURO SOCIAL',
 'efr': 'UNIÃO',
 'uf': 'CE',
 'telefone': '(85) 3255-7401',
 'email': 'gexfor@inss.gov.br',
 'situacao': 'ATIVA',
 'bairro': 'CENTRO',
 'logradouro': 'R PEDRO PEREIRA',
 'numero': '383',
 'cep': '60.035-000',
 'municipio': 'FORTALEZA',
 'porte': 'DEMAIS',
 'abertura': '26/05/1978',
 'natureza_juridica': '110-4 - Autarquia Federal',
 'fantasia': 'GERENCIA EXECUTIVA FORTALEZA',
 'cnpj': '29.979.036/0042-19',
 'ultima_atualizacao': '2021-04-01T08:41:40.572Z',
 'status': 'OK',
 'complemento': '',
 'motivo_situacao': '',
 'situacao_especial': '',
 'data_situacao_especial': '',
 'atividades_secundarias': [{'code': '00.00-0-00', 'text': 'Não informada'}],
 'capital_social': '0.00',
 'qsa': [],
 'extra': {},
 'billing': {'free': True, 'database': True}}

In [16]:
# Adicionar situacao da Empresa ao dataset
for empenho in empenhos:
#     print(empenho["numero_documento_negociante"])
    
    cpfCnpj = str(empenho["numero_documento_negociante"]).strip()
    
    if len(cpfCnpj) == 14:
        favorecido = fetch_cnpj_info(cpfCnpj)
        if favorecido != None:
            empenho["situacao_favorecido"] = favorecido["situacao"]
            empenho["atividade_principal_texto"] = favorecido["atividade_principal"][0]["text"]
            empenho["atividade_principal_codigo"] = favorecido["atividade_principal"][0]["code"]
        else:
            empenho["situacao_favorecido"] = ""
            empenho["atividade_principal"] = ""

empenhos[:1]

[{'codigo_municipio': '014',
  'exercicio_orcamento': '201600',
  'codigo_orgao': '06',
  'codigo_unidade': '02  ',
  'data_emissao_empenho': '2016-01-04 00:00:00',
  'numero_empenho': '04010307',
  'data_referencia_empenho': '201601',
  'codigo_funcao': '13',
  'codigo_subfuncao': '392',
  'codigo_programa': '0064',
  'codigo_projeto_atividade': '2',
  'numero_projeto_atividade': '043',
  'numero_subprojeto_atividade': '0000',
  'codigo_elemento_despesa': '33903000',
  'modalidade_empenho': 'O',
  'descricao_empenho': 'Valor que se empenha p/ fazer face as despesas com AQUISICAO DE COLCHOES DESTINADOS A SER UTILIZADO PELOS POLICIAIS QUE VAO FAZER A SEGURANCA DO CARNAVAL CULTURAL 2016 NO MUNICIPIO DE ARACATI.',
  'valor_anterior_saldo_dotacao': '8500.0000',
  'valor_empenhado': '5980.0000',
  'valor_atual_saldo_dotacao': '2520.0000',
  'tipo_processo_licitatorio': 'F',
  'numero_documento_negociante': '41426966005050',
  'estado_empenho': 'OR',
  'numero_nota_anulacao': ' ',
  'data_em

### Criar CSV

In [17]:
cabecalhos = empenhos[0].keys()
cabecalhos

dict_keys(['codigo_municipio', 'exercicio_orcamento', 'codigo_orgao', 'codigo_unidade', 'data_emissao_empenho', 'numero_empenho', 'data_referencia_empenho', 'codigo_funcao', 'codigo_subfuncao', 'codigo_programa', 'codigo_projeto_atividade', 'numero_projeto_atividade', 'numero_subprojeto_atividade', 'codigo_elemento_despesa', 'modalidade_empenho', 'descricao_empenho', 'valor_anterior_saldo_dotacao', 'valor_empenhado', 'valor_atual_saldo_dotacao', 'tipo_processo_licitatorio', 'numero_documento_negociante', 'estado_empenho', 'numero_nota_anulacao', 'data_emissao_empenho_substituto', 'numero_empenho_substituto', 'cd_cpf_gestor', 'cpf_gestor_contrato', 'data_contrato', 'numero_contrato', 'data_realizacao_licitacao', 'numero_licitacao', 'codigo_tipo_negociante', 'nome_negociante', 'endereco_negociante', 'fone_negociante', 'cep_negociante', 'nome_municipio_negociante', 'codigo_uf', 'tipo_fonte', 'codigo_fonte', 'situacao_favorecido', 'atividade_principal_texto', 'atividade_principal_codigo'])

In [18]:
# Cria o arquivo.
arquivo_csv = open('empenhos_tce.csv', 'w', newline='', encoding='utf-8')

# Cria o objeto de gravação
escrever = csv.writer(arquivo_csv, delimiter=',')

# Grava os cabecalhos
escrever.writerow(cabecalhos)

for linha in empenhos:
    escrever.writerow(linha.values())

# Fecha o arquivo.
arquivo_csv.close()

In [19]:
df = pd.read_csv("empenhos_tce.csv")
df.head()

,codigo_municipio,exercicio_orcamento,codigo_orgao,codigo_unidade,data_emissao_empenho,numero_empenho,data_referencia_empenho,codigo_funcao,codigo_subfuncao,codigo_programa,...,endereco_negociante,fone_negociante,cep_negociante,nome_municipio_negociante,codigo_uf,tipo_fonte,codigo_fonte,situacao_favorecido,atividade_principal_texto,atividade_principal_codigo
0,14,201600,6,2,2016-01-04 00:00:00,4010307,201601,13,392,64,...,RUA CEL ALEXANDRINO 1748 LOJA 01 CACIMBA DO...,08834219250,62800000,Aracati,CE,1,1,ATIVA,Comércio varejista de móveis,47.54-7-01
1,14,201600,6,2,2016-01-04 00:00:00,4010451,201601,13,122,45,...,"RUA CEL. ALEXANDRINO, 860 CENTRO",,62800000,Aracati,CE,1,1,ATIVA,"Bancos múltiplos, com carteira comercial",64.22-1-00
2,14,201600,6,2,2016-01-08 00:00:00,8010002,201601,13,122,45,...,"Rua Dragao do Mar, 33 CENTRO",,62800000,Aracati,CE,1,1,ATIVA,"Serviços de organização de feiras, congressos,...",82.30-0-01
3,14,201600,6,2,2016-01-11 00:00:00,11010003,201601,13,392,64,...,"RUA CEL. ALEXANDRINO, 680 CENTRO",08834211330,62800000,Aracati,CE,1,1,INAPTA,********,00.00-0-00
4,14,201600,6,2,2016-01-15 00:00:00,15010002,201601,13,392,64,...,"RUA VILA SAO CRISTOVAO, 964 NS DE LOURDES",,62800000,Aracati,CE,1,1,ATIVA,Atividades de organizações associativas ligada...,94.93-6-00


In [20]:
df.shape

(889, 43)